In [1]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
import math
from tensorboardX import SummaryWriter

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

REPORT_EVERY_ITER = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device', device)

importing Jupyter notebook from CustomDataset.ipynb
importing Jupyter notebook from Model.ipynb
units after conv 512
conv parameters:  168224
fc parameters:  166933
using device cuda


In [2]:
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [3]:
dataset = ControlsDataset()
dataset.convertTOClass()
print("Stack size", dataset.stack_size)
# dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)
net = ConvNet().to(device)

print("Data size", dataset.data_frame.size)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

Stack size 1
units after conv 512
conv parameters:  168224
fc parameters:  166933
Data size 22038
number of parameters:  335157


In [4]:
print(dataset.data_frame.head(10))

   ID  Angle
0   0     10
1   1     10
2   2     10
3   3     10
4   4     10
5   5     10
6   6     10
7   7     10
8   8     10
9   9     10


In [5]:
validation_split = .2
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

# spliting the dataset
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


# Training data loader # NOTE had to remove shuffle
dataloader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=train_sampler)

# Validation data loader # NOTE had to remove shuffle
validLoader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=valid_sampler)

print("Total training stacks", len(dataloader))
print("Total validation stacks",len(validLoader))

Total training stacks 35
Total validation stacks 9


In [6]:
epochs = 25
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss() # Changed from Mean-Squared to Cross-Entropy
writer = SummaryWriter()
iter_no = 0

torch.cuda.empty_cache()
for epoch in range(epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        iter_no += 1
        images = sampled_batch['image'].to(device).float()
        controls = sampled_batch['control'].to(device).long()
        #print("imagesSize: ", images.size(), "controlsSize:", controls.size())
        optimizer.zero_grad()
        prediction = net(images)
        
        controls = torch.flatten(controls)
        loss = criterion(prediction, controls)
        
        #prints the categories that have the highest value for each item in the batch
        print(torch.argmax(prediction,axis=1))
        #prints the coresponding targets
        print(controls)
        
        loss.backward()
        optimizer.step()

        if iter_no % REPORT_EVERY_ITER == 0:
            writer.add_scalar("Loss", loss.item(), iter_no)
            torch.save(net.state_dict(), "snapshots/saved_model_{}".format(loss.item()))
        
        out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
        print(out.format(str(epoch),
                        str(iter_no),
                        round(loss.item(),5),
                         #'na', 'na'
                         round(torch.cuda.memory_allocated(0)/1024**3,3),
                         round(torch.cuda.memory_allocated(0)/1024**3,3)
                        ))
        #torch.cuda.empty_cache()

tensor([17, 15, 17, 12, 17, 17, 19, 12, 17, 15, 19, 17, 17,  4, 17, 15, 17, 17,
        17, 15, 19, 15, 15, 17, 12, 17, 15, 17, 17, 12, 12, 17, 17, 15, 15, 17,
        17, 12, 19, 17, 15, 15, 19, 12, 17, 15, 17, 15, 19, 17, 15, 19, 17, 15,
        19, 17, 17,  1, 17, 17, 19, 17, 12, 17, 19, 19, 17, 17, 17, 17, 15, 17,
        17, 15, 17, 17, 17, 15, 17, 15, 12, 17, 15, 17, 17, 17, 17, 17,  1, 17,
        15, 15, 17, 15, 17, 17, 19, 17, 15, 15, 17, 17, 17,  1, 15, 17, 17, 19,
         4, 17, 15, 17, 17,  1, 17, 17, 15, 17, 17, 12, 12, 15, 17, 15, 15, 12,
        17, 17, 15, 15, 17, 19, 17, 17, 17, 17, 17, 17, 19, 17, 17, 15, 12, 15,
        19, 17, 17, 17, 15, 17, 15, 17, 19, 15, 12, 17, 17, 17, 17, 12, 17, 17,
        15, 19, 12, 17, 15, 12, 15, 15, 17, 15, 17, 15, 17, 17, 19, 17, 17, 17,
        17, 19, 17, 12, 17, 15, 15, 15, 15, 15, 17, 15, 15, 17, 15, 17, 17, 17,
        15,  1, 17, 17, 17, 17, 12, 17, 17, 17, 19, 15, 17, 17, 17, 15, 15, 17,
        17, 19, 17, 17, 17, 17, 15, 17, 

0,4	Loss:1.85045	Allocated:0.884GB	Cached:0.884GB

tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1

0,8	Loss:1.72426	Allocated:0.884GB	Cached:0.884GB

tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1

0,12	Loss:1.61525	Allocated:0.884GB	Cached:0.884GB

tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 

KeyboardInterrupt: 

In [ ]:
# Validation Test
iter_no = 0
total = 0
correct = 0
for i_batch, sampled_batch in enumerate(validLoader):
    iter_no += 1
    images = sampled_batch['image'].to(device).float()
    controls = sampled_batch['control'].to(device).long()
    controls = torch.flatten(controls)
    optimizer.zero_grad()
    prediction = net(images)
    
    maximum = torch.argmax(prediction,axis = 1)
    print(maximum)
    print(controls)
    shared = maximum == controls
    shared = 1 * shared
    correct += int(torch.sum(shared))
    print(correct)
    total += len(controls)
    print(correct/total)

In [ ]:
print(total)
print(correct)
print("Accuracy: {}".format(correct/total))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))